<a href="https://colab.research.google.com/github/nekoniii3/Image_Upscaling/blob/main/Image_Upscaling_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# パッケージのインストール
!pip install gradio
!pip install stability-sdk

In [ ]:
import io
import os
import warnings
import gradio as gr

from IPython.display import display
from PIL import Image
from stability_sdk import client
import stability_sdk.interfaces.gooseai.generation.generation_pb2 as generation

# ラップする関数
def upscale(img, key, engine):

  # Set up our connection to the API.
  stability_api = client.StabilityInference(
      key=key, # API Key reference.
      upscale_engine=engine,
      verbose=True # Print debug messages.
  )

  answers = stability_api.upscale(
      init_image=img
  )

  for resp in answers:
      for artifact in resp.artifacts:
          if artifact.finish_reason == generation.FILTER:
              warnings.warn(
                  "Your request activated the API's safety filters and could not be processed."
                  "Please submit a different image and try again.")
          if artifact.type == generation.ARTIFACT_IMAGE:
              big_img = Image.open(io.BytesIO(artifact.binary))

  return big_img

# 入出力定義
demo = gr.Interface(
    fn=upscale, 
    inputs=[gr.Image(type="pil")
    , gr.Textbox(type='password' , placeholder="stability key here...")
    , gr.Dropdown(  ["esrgan-v1-x2plus", "stable-diffusion-x4-latent-upscaler"], value="esrgan-v1-x2plus", label="engine")
    ],
    outputs="image"
    , allow_flagging='never'
)

# 起動
demo.launch()